# 1. Giới thiệu
Mercari là một công ty thương mại điện tử của Nhật Bản được thành lập vào tháng 2 năm 2013 và hiện đang hoạt động tại Nhật Bản và Hoa Kỳ. Sản phẩm chính của Mercari là 'Mercari marketplace app' được ra mắt lần đầu tiên tại Nhật Bản vào tháng 7 năm 2013 và kể từ đó đã phát triển trở thành thị trường do cộng đồng hỗ trợ lớn nhất Nhật Bản. Ứng dụng này muốn xây dựng một mô hình có thể đưa ra các đề xuất về giá cho người bán hoặc thậm chí cho người mua để biết liệu họ có đang mua sản phẩm với giá chính xác hay không.

# 2. Mục tiêu
Mục tiêu ở đây là đưa ra một thuật toán tự động dự đoán giá của một sản phẩm một cách phù hợp dựa trên tập dữ liệu được cung cấp bởi Mercari

# 3. Mục lục
- Cài đặt dữ liệu
- Giải nén dữ liệu
- Chuẩn bị thư viện
- Đọc dữ liệu
- Phân tích và xử lí dữ liệu
- Mô hình hóa dữ liệu
- Đánh giá mô hình
- Dự đoán
- Submission

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 4. Cài đặt dữ liệu

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

# 5. Giải nén dữ liệu

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

# 6. Chuẩn bị thư viện
import một số thư viện để sử dụng

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

# 7. Đọc dữ liệu

In [ ]:
#Đọc dữ liệu từ 2 tập 'train.tsv' và tập 'test_stg2.tsv'(đưa về dạng bảng)
train = pd.read_table('train.tsv')
test = pd.read_table('test_stg2.tsv')

In [ ]:
#Quan sát dữ liệu ở tập train
#Sử dụng dữ liệu ở tập train để huấn luyện mô hình
train.head()

**Quan sát:**
- Ở tập train, mỗi 1 sản phấm sẽ có các thuộc tính: train_id, name, item_condition_id, category_name, shipping, item_description và cột price
- Ở cột brand_name có nhiều giá trị NaN(giá trị rỗng)

=> kiểm tra các giá trị trong tập 'train'

In [ ]:
#Kiểm tra thông tin của tập 'train'
train.info()

In [ ]:
#Quan sát dữ liệu ở tập 'test'
#Sử dụng sử dụng tập test để thử nghiệm mô hình
test.head()

**Quan sát:**
- Mỗi một sản phẩm trong tập 'test' bao gồm các thuộc tính: test_id, name, item_condition_id, category_name, brand_name, shipping, item_description 
- Không có cột 'price' vì ta sẽ dùng 1 mô hình nào đó để đoán giá của mỗi sản phẩm

In [ ]:
#Kiểm tra thông tin dữ liệu ở tập 'test'
test.info()

# 8. Xử lí và phân tích dữ liệu

**Mục tiêu**: Ở phần này chúng ta sẽ xử lí các dữ liệu. Vì bài toán này là bài toán hồi quy tuyến tính nên cần chuyển các thuộc tính về dạng vector hoặc số thực

Như đã quan sát ở trên ta thấy rằng ở tập 'train' và tập 'test' đều chứa các giá trị rỗng nên ta cần phải kiểm tra các giá trị rỗng này

# 8.1 Quan sát tổng quan

In [ ]:
#sử dung hàm isnull() đếm các giá trị null
print("train: ")
print(pd.isnull(train).sum())
print("="*30)
print("test: ")
print(pd.isnull(test).sum())

=> chỉ có 2 thuộc tính có giá trị null là category_name và brand_name

In [ ]:
print("{0:.2f}% sản phẩm không có thương hiệu(brand_name) ở tập train".format((pd.isnull(train['brand_name']).sum())*100/train.shape[0]))

print("{0:.2f}% sản phẩm không có phân loại(category_name) ở tập train".format((pd.isnull(train['category_name']).sum())*100/train.shape[0]))

In [ ]:
print("{0:.2f}% sản phẩm không có thương hiệu(brand_name) ở tập test".format((pd.isnull(test['brand_name']).sum())*100/test.shape[0]))

print("{0:.2f}% sản phẩm không có thương hiệu(brand_name) ở tập test".format((pd.isnull(test['category_name']).sum())*100/test.shape[0]))

**=> Quan sát dữ liệu ở trên ta thấy được:**
- Ở tập dữ liệu train và test, cột category_name có nhiều cấp được phân tách bằng "/"
- Có 2 tập có giá trị null
- brand_name có gần một nửa giá trị bị thiếu cả trong tập train và tập test, vì vậy không thể xóa những hàng mà phải lấp các giá trị đó
- trong khi cột category_name thiếu 0,43-0,44% giá trị nên có thể xóa các hàng đó hoặc lấp đầy các giá trị đó


In [ ]:
import gc
gc.collect()

# 8.2 Xử lí các giá trị rỗng

Việc tồn trại các giá trị rỗng sẽ làm cho đánh giá không chính xác

**Lấp các giá trị null ở cột brand_name**

In [ ]:
# Lấp các giá trị null bằng 'Nobrand'
train['brand_name'] = train['brand_name'].fillna('Nobrand')
test['brand_name'] = test['brand_name'].fillna('Nobrand')

**Lấp các giá trị null ở cột category_name**

In [ ]:
# Lấp các giá trị null bằng 'No/No/No'
train['category_name'] = train['category_name'].fillna('No/No/No')
test['category_name'] =test['category_name'].fillna('No/No/No')

**Lấp các giá trị null ở cột item_description**

In [ ]:
#Lấp các giá trị null bằng 'No description'
train['item_description'] = train['item_description'].fillna('No Description')
test['item_description'] = test['item_description'].fillna('No Description')

In [ ]:
#Quan sát dữ liệu ở tập tain sau khi lấp các giá trị null
train.head(10)

**Quan sát:**

- Quan sát 10 sản phẩm đau tiên của tập 'train' các giá trị rống đã được thay thế bằng các text phù hợp

In [ ]:
#Kiểm tra các giá trị rỗng
print(pd.isnull(train).sum())
print("="*30)
print(pd.isnull(test).sum())


=> Không còn giá trị null

**Sử dụng LabelBinarizer để xử lí các dữ liệu dạng label và Vectorizer để xử lí các dữ liệu dạng text**

Vì đây là mô hình hồi quy nên cần phải đưa dữ liệu về dạng vecto để xử lí

# 8.3 Xử lí dữ liệu ở cột category_name

In [ ]:
#kiểm tra category_name
levels =[]

for values in train['category_name']:
    levels.append(values.count("/"))


print("MIN  of levels:          {0:.0f} ".format(np.min(levels)+1))
print("MEDIAN of levels:        {0:.0f} ".format(np.percentile(levels, 50)+1))
print("MAX  of levels:          {0:.0f}".format(np.max(levels)+1))
print("90 percentile  of levels:{0:.0f} ".format(np.percentile(levels, 90)+1))

**Quan sát**
- category_name có tối đa 5 mục được phân tách bằng "/"
- category_name có tối thiểu 3 mục "/"
- 90% có 3 mục
=> Chia category_name thành 3 cột riêng biệt


**Chia cột category_name thành cat_1, cat_2, cat_3**

=> việc này giúp cho mô hình huấn luyện chính xác hơn

In [ ]:
#Hàm split_cat dùng để cắt các nhãn trong cột 'category_name' thành các cột riêng biệt
def split_cat(category_name):
    try:
        return category_name.split('/')
        return ("done")
    except:
        return ['Null', 'Null', 'Null']

In [ ]:
train['cat_1'], train['cat_2'], train['cat_3'] = zip(*train['category_name'].apply(lambda x:split_cat(x)))

test['cat_1'], test['cat_2'], test['cat_3'] = zip(*test['category_name'].apply(lambda x: split_cat(x)))
# Bởi vì có những list of list nên phải sử dụng zip (*) để có được danh sách 3 cuối cùng

In [ ]:
train.head()

In [ ]:
test.head()

=> sau khi chia thì trong dữ liệu trong tập 'train' và tập 'test' có thêm 3 cột 'cat_1', 'cat_2', 'cat_3'

# 8.4 Quan sát và phân tích các dữ liệu dạng số
- Các cột thuộc dữ liệu dạng số bao gồm: train_id, item_condition_id, price, shipping

# 8.4.1 Quan sát tổng quan

In [ ]:
#Định dạng 
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#Thống kê số lượng, giá trị trung bình, min, max, giá trị phổ biến của các cột chứa dữ liệu dạng số
train.describe()

**Quan sát**
- Ở đây chỉ có 2 cột dữ liệu dùng để phân tích dự đoán là cột item_condition_id và shipping
- Giá trị của các mặt hàng nhỏ nhất là không tức là có những sản phẩm có thể xem như là miễn phí và lớn nhất là 2009
- Giá trị phổ biến của các sản phẩm dao động từ 17 $ - 2009 $
- Phần lớn các mặt hàng có shipping = 1

# 8.4.2 Đánh giá cột price

In [ ]:
#Biểu đồ phân bố giá của các sản phẩm trong tập train)
plt.figure(figsize=(10,5))
plt.hist(train['price'], bins = 100, edgecolor='white', range=[0,300])
plt.ylabel('Frequency')
plt.xlabel('Price')
plt.show()

**Quan sát:**
- chúng ta có thể thấy rằng phân phối ở trên bị lệch về bên trái

=> Đưa về phân phối chuẩn với log(price)

In [ ]:
#Biểu đồ phân phối log(price)
plt.figure(figsize=(10,5))
plt.hist(np.log(train['price']+1),bins=50, edgecolor='white') 
plt.ylabel('Frequency')
plt.xlabel('log_Price')
plt.show()

**Quan sát**
- Phân bố của log(price) ít lệch hơn và được phân bố tốt xung quanh giá trị trung bình

=> Sử dụng log(price) thay vì price để huấn luyện

# 8.4.3 Quan sát và đánh giá dữ liệu ở cột item_condition_id

In [ ]:
#Biểu đồ phân tích cột item_condition_id
ax = sns.countplot(x = 'item_condition_id',data=train, palette ='Blues_r')
ax.set_title("Tổng số lượng sản phẩm theo item_condition_id", fontsize = 13)

**Quan sát**
- Có 5 loại tình item_condition_id
- Loại 1, loại 2 và loại 3 là phổ biến nhất
- Loại 4 và loại 5 là ít phổ biến nhất

# 8.4.4 Quan sát cà đánh giá dữ liệu ở cột shipping

In [ ]:
(train['shipping'].value_counts())*100/train.shape[0]

=> Có khoảng 55% khách hàng không phải trả phí vận chuyển

In [ ]:
plt.figure(figsize=(10,8))

plt.hist(np.log(train['price'][train['shipping']==0]+1),bins=50, edgecolor='white',color="blue", label='shipping = 0') 
plt.hist(np.log(train['price'][train['shipping']==1]+1),bins=50, edgecolor='white',color="cornflowerblue",label="shipping = 1") 
plt.xlabel('log_Price')
plt.legend(loc='upper right')
plt.title('Shipping')
plt.show()

**Quan sát:**
- Giá trị trung bình của những sản phẩm có shipping = 0 nhỏ hơn giá trị trung bình của những sản phẩm có shipping = 1.

# 8.5 Quan sát dữ liệu ở cột brand_name

In [ ]:
#danh sách 10 thuong hiệu phổ biến nhất
brands =train['brand_name'].value_counts()
print(brands[:10])

**Quan sát:**
- Những sản phẩm khoogn có thương hiệu chiếm số lượng nhiều nhất.
- Các thương hiệu phổ biến nhất là: PINK, Nike, Victoria's Secret, LuLaRoe, Apple, FOREVER 21, Nintendo, Lululemon, Michael Kors.

In [ ]:
#Quan sát phân phối giá của các thương hiệu phổ biến nhất
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 15))
plt.subplot(3, 3, 1)#hình có 3 hàng 3 cột và đây là biểu đò đầu tiên trong hình
plt.hist(np.log(train['price'][train['brand_name']=='Nobrand']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Nobrand')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=='Nike']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Nike')

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=="Victoria's Secret"]+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title("Victoria's Secret")

plt.subplot(3, 3, 4)
plt.hist(np.log(train['price'][train['brand_name']=='LuLaRoe']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('LuLaRoe')

plt.subplot(3, 3, 5)
plt.hist(np.log(train['price'][train['brand_name']=='Apple']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Apple')

plt.subplot(3, 3, 6)
plt.hist(np.log(train['price'][train['brand_name']=='FOREVER 21']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('FOREVER 21')

plt.subplot(3, 3, 7)
plt.hist(np.log(train['price'][train['brand_name']=='Nintendo']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Nintendo')

plt.subplot(3, 3, 8)
plt.hist(np.log(train['price'][train['brand_name']=='Lululemon']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Lululemon')

plt.subplot(3, 3, 9)
plt.hist(np.log(train['price'][train['brand_name']=='Michael Kors']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Michael Kors')

plt.show()

**Quan sát**
- Phân phối của log(price) của các thương hiệu phổ biến và các sản phẩm không có thương hiệu đều tuân theo phân phối chuẩn

In [ ]:
#một số thương hiệu ít phổ biến hơn
print(brands[250:260])

In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(3, 3, 1)
plt.hist(np.log(train['price'][train['brand_name']=='Sperrys']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Sperrys')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=='Disney Collection']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Disney Collection')

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=='Madewell']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Madewell')

plt.subplot(3, 3, 4)
plt.hist(np.log(train['price'][train['brand_name']=='Mudd']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Mudd')

plt.subplot(3, 3, 5)
plt.hist(np.log(train['price'][train['brand_name']=='7 For All Mankind®']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('7 For All Mankind®')

plt.subplot(3, 3, 6)
plt.hist(np.log(train['price'][train['brand_name']=='Gerber']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Gerber')

plt.subplot(3, 3, 7)
plt.hist(np.log(train['price'][train['brand_name']=='Diamond Supply Co.']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Diamond Supply Co.')

plt.subplot(3, 3, 8)
plt.hist(np.log(train['price'][train['brand_name']=='Leap Frog']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Leap Frog')

plt.subplot(3, 3, 9)
plt.hist(np.log(train['price'][train['brand_name']=='BCBGMAXAZRIA']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('BCBGMAXAZRIA')

plt.show()

**Quan sát**
- Đa số các phân phối log_Price của các hãng này đều tuân theo phân phối chuẩn

In [ ]:
#10 thương hiệu ít phổ biến nhất
print(brands[-10:-1])

# 8.6 Quan sát dữ liệu ở các cột  cat_1, cat_2 và cat_3

# 8.6.1 'cat_1'

In [ ]:
print("Có %d nhãn ở cột cat_1." % train['cat_1'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x = 'cat_1', data = train, palette ='Blues_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng các sản phẩm chứa các nhãn ở cột cat_1')


=> Phân loại hàng Women, beauty là nhiều nhất

**Quan sát các nhãn ở cat_1 tương quan với giá**

In [ ]:
train.groupby('cat_1')['price'].describe()

# 8.6.2 'cat_2'

In [ ]:
print("Có %d nhãn ở cột cat_2." % train['cat_2'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x='cat_2',data=train,order=train.cat_2.value_counts().iloc[:20].index, palette ='Reds_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng sản phẩm chứa các nhãn ở cat_2 ')

Quan sát một số sản phẩm chứa các nhãn ở cat_2 tương quan với giá

In [ ]:
train.groupby('cat_2')['price'].describe()[:10]

# 8.6.3 'cat_3'

In [ ]:
print("Có %d nhãn ở cột cat_3." % train['cat_3'].nunique())

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(x='cat_3',data=train,order=train.cat_3.value_counts().iloc[:20].index, palette ='Greens_r', edgecolor='k')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Biểu đồ số lượng sản phẩm chứa các nhãn ở cat_3 ')

Quan sát một số sản phẩm chứa các nhãn ở cat_2 tương quan với giá

In [ ]:
train.groupby('cat_3')['price'].describe()[:10]

# 8.7 Quan sát dữ liệu ở cột 'name'

In [ ]:
# Đếm số lượng 1 số từ trong 'name'
train['name'].value_counts()[:10]

In [ ]:
x = train['name'].apply(lambda x: len(x))
plt.hist(x,bins = 30,range=[0,50],edgecolor='white')
plt.show()

**Quan sát:**
- Tên của mỗi sản phẩm có khoangr từ 1 đến 42 từ và chủ yếu nẳm trong khoảng từ 15 đến 40 từ

# 8.8 Quan sát dữ liệu ở cột 'item_description'

In [ ]:
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(train.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

=> Các cụm từ "Brand new", "free shipping", "great condition", " good condition", "never worn", "smoke free", " description yet" là những cụm từ xuất hiện nhiều nhất

# 9. Vecto hóa dữ dữ liệu

**Mục tiêu:** Đưa dữ liệu về dạng vecto để huấn luyện theo mô hình hồi quy tuyến tính

[Tham khảo về mô hình hồi quy tuyến tính](https://vi.wikipedia.org/wiki/H%E1%BB%93i_quy_tuy%E1%BA%BFn_t%C3%ADnh)

# 9.1: Vecto hóa dữ liệu dạng label

**Sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân**

- Dữ liệu trả về là sparse matrix (ma trận thưa)

**[item_condition_id]**

In [ ]:
lb_item_condition_id = LabelBinarizer(sparse_output=True)
train_condition = lb_item_condition_id.fit_transform(train['item_condition_id'])
test_condition = lb_item_condition_id.transform(test['item_condition_id'])

In [ ]:
train_condition.shape

=> ma trận trả về có 1482535 hàng (vì trong tập train có dữ liệu của 1482535 sản phẩm) và có 5 cột vì chỉ có 5 kiểu item_condition

**[shipping]**

In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)
train_shipping = lb_shipping.fit_transform(train['shipping'])
test_shipping = lb_shipping.transform(test['shipping'])

In [ ]:
train_shipping.shape

=> ma trận trả về có 1482535 hàng (vì trong tập train có dữ liệu của 1482535 sản phẩm) và có 1 cột vì chỉ có 2 kiểu shipping

**[brand_name]**

In [ ]:
lb_brand_name = LabelBinarizer(sparse_output=True)
train_brand_name= lb_brand_name.fit_transform(train['brand_name'])
test_brand_name = lb_brand_name.transform(test['brand_name'])


In [ ]:
train_brand_name.shape

=> ma trận trả về có 1482535 hàng (vì trong tập train có dữ liệu của 1482535 sản phẩm) và có 4810 cột vì  có 4810 thương hiệu

**[name]**

Vì mỗi một chuỗi ở cột "name" khá ngắn 

=>

-  Sử dụng CountVectorizer() để chuyển cột name từ dạng text về dạng một vecto trên cơ sở số lần xuất hiện của mỗi từ
- CountVectorizer tạo một ma trận trong đó mỗi từ duy nhất được biểu thị bằng một cột của ma trận và với tên của mỗi sản phẩm  là một hàng trong ma trận. Giao của một hàng và một cột chính là số lần xuất hiện của từ(tương úng với cột) trong tên của sản phẩm (tương úng với hàng)

In [ ]:
count_vec = CountVectorizer()

train_name = count_vec.fit_transform(train['name'])
test_name = count_vec.transform(test['name'])

In [ ]:
print(train_name.shape)

=> ma trận 'train_name'có 1482535 hàng là vì có 1482535 sản phẩm trong tập 'train' và có 105757 tức là có tất cả 105757 từ xuất hiện trong cột 'name'

**[item_description]**

Do số lượng từ ở cột item_description nhiều
=> sử dụng TfidfVectorizer() để chuyển đổi dữ liệu thành dạng ma trận

TF-IDF (Term Frequency – Inverse Document Frequency) là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản.

TF: Term Frequency(Tần suất xuất hiện của từ) là số lần từ xuất hiện trong văn bản. Vì các văn bản có thể có độ dài ngắn khác nhau nên một số từ có thể xuất hiện nhiều lần trong một văn bản dài hơn là một văn bản ngắn. Như vậy, term frequency thường được chia cho độ dài văn bản( tổng số từ trong một văn bản).

IDF: Inverse Document Frequency(Nghịch đảo tần suất của văn bản), giúp đánh giá tầm quan trọng của một từ . Khi tính toán TF , tất cả các từ được coi như có độ quan trọng bằng nhau. Nhưng  một số từ như “is”, “of” và “that” thường xuất hiện rất nhiều lần nhưng độ quan trọng là không cao. Như thế chúng ta cần giảm độ quan trọng của những từ này xuống.

In [ ]:
tfidf_des = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), stop_words='english')

train_des = tfidf_des.fit_transform(train['item_description'])
test_des = tfidf_des.transform(test['item_description'])

In [ ]:
train_des.shape

Sau khi chuẩn hóa dữ liệu ở các cột category_name, sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân ở các cột cat_1, cat_2, cat_3

In [ ]:
lb_cat_1 = LabelBinarizer(sparse_output=True)
train_cat_1 = lb_cat_1.fit_transform(train['cat_1'])
test_cat_1 = lb_cat_1.transform(test['cat_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
train_cat_2 = lb_cat_2.fit_transform(train['cat_2'])
test_cat_2 = lb_cat_2.transform(test['cat_2'])

lb_cat_3 = LabelBinarizer(sparse_output=True)
train_cat_3 = lb_cat_3.fit_transform(train['cat_3'])
test_cat_3 = lb_cat_3.transform(test['cat_3'])

# 10. Mô hình hóa dữ liệu

 **Tạo sparse matrix để kết hợp các dữ liệu với nhau**

In [ ]:
from scipy.sparse import hstack
import gc

In [ ]:
sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)

In [ ]:
X_train = hstack(sparse_matrix_list).tocsr()
print( X_train.shape)

del X_train
gc.collect()

# 11. Đánh giá mô hình

# 11.1 Số liệu đánh giá
**RMSLE**
- Số liệu đánh giá cuộc thì này là [Root Mean Squared Logarithmic Error](https://www.kaggle.com/c/mercari-price-suggestion-challenge/overview/evaluation)
- Lỗi trung bình bình phương (RMSE) là độ lệch chuẩn của phần dư ( lỗi dự đoán ). Phần dư là thước đo khoảng cách từ các điểm dữ liệu đường hồi quy; RMSE là thước đo mức độ lan truyền của những phần dư này. Nói cách khác, nó cho bạn biết mức độ tập trung của dữ liệu xung quanh dòng phù hợp nhất . Lỗi bình phương trung bình thường được sử dụng trong khí hậu học, dự báo và phân tích hồi quy để xác minh kết quả thí nghiệm.
- Công thức tính RMSLE

![alt](https://thienmaonline.vn/rmse-la-gi/imager_5_11689_700.jpg)

Trong đó: 
+ y^i là giá trị ước lượng
+ yi là biến độc lập
+ n=(N – k – 1)
+ N : số tổng lượng quan sát
+ K : tổng lượng biến


**Tính rmsle**

In [ ]:
#Tính rmsle
def rmsle(y, y_preds):
    assert len(y) == len(y_preds)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_preds), 2)))

In [ ]:
from sklearn.model_selection import train_test_split

def run_model(model, matrix_list):
    #X= train[['item_condition_id','shipping','name','brand_name','cat_1','cat_2','cat_3']]
    #X_train, x_test, Y_train, y_test = train_test_split(X, train['price'], test_size=0.25)
    X = hstack(matrix_list).tocsr()
    X_train, x_test, Y_train, y_test = train_test_split(X, np.log1p(train['price']), test_size=0.2)
    model.fit(X_train, Y_train)
    preds = model.predict(x_test)
    del X, X_train, x_test, Y_train
    gc.collect()
    
    return preds, y_test

 # 11.2 Model 1: Ridge Regression

**Tổng quan:**

- Mô hình Ridge Regression là phương pháp khi áp dụng khi bộ dữ liệu gặp vấn đề về đa cộng tuyến tính ( các biến độc lập x có mối liên hệ với nhau và ảnh hưởng lên kết quả dự đoán của y, hay giải quyết các vấn đề về Overfitting (mô hình áp dụng tốt cho mô hình training nhưn không hoạt động tốt trên dữ liệu test) và các mô hình hồi quy tuyến tính thường gặp.
- Phương trỉnh tổng quát của linear regression cho đơn biến và đa biến có các giá trị εở đằng sau mỗi phương trình. Đây là sai số của các phương trình hồi quy, là chênh lệch giữa kết quả dự báo và kế quả thực tế. Các sai số được chia làm 2 phần: Biased (thiên vị), Variance ( phương sai). Bias là trương hợp mô hình phân tích không khớp, không đem lại kết quả không chính xác trên tập dữ liệu training, còn Variance là đối với tập dữ liệu test. Mối quan hệ đánh đổi giữa Biased  và Variance xét trên mức độ ohuwcs tạp của mô hình

>![alt](https://i.stack.imgur.com/w0ldM.png)
- Ridge Regression là mô hình hồi quy phân tích mối quan hệ giữa các biến độc lập và các biến phụ thuộc sử dụng phươn pháp Regularization, điều chình mô hình sao cho giảm thiểu các vấn đề Overfitting, tối uuwu hay kiểm soát mức độ phức tạp của mô hình để cân đối giữa Biased và Variance Quan đó giảm sai số của mô hình.
- Công thức tổng quát của mô hình:


 >![alt](https://www.einfochips.com/blog/wp-content/uploads/2019/01/L2-Regularization.png)

Hệ số λ hay còn gọi là tham số Regularization, hay tham số Penalty, hay tham số Srinkage, là số luôn dương, là giá trị mà ở đó phương trình tuyến tính sẽ được "tinh chỉnh" sao cho sai số của mô hình được giảm tối đa, nghĩa là giá trị λ nào làm cho MSE (ở đây dung RMSE) nhỏ nhất thì mô hình đó được chọn, Wi là hệ số β của phương trình hồi quy tuyến tính.
- Đồ thị minh họa:

>![alt](https://i.stack.imgur.com/w0ldM.png)




In [ ]:
from sklearn.linear_model import Ridge
model = ridge = Ridge()
Ridge_preds, y_test = run_model(model, matrix_list=sparse_matrix_list)
#print(evaluate_orig_price(y_test, Ridge_preds))

In [ ]:
print("rmsle: "+str(rmsle(np.expm1(y_test), np.expm1(Ridge_preds))))

# 11.3 Model 2: LGBM Regression

**Tổng quan**
- LGBM (Light Gradient Boosting Machine) là một khung tăng cường độ dốc dựa trên cây quyết định để tăng hiệu quả của mô hình và giảm mức sử dụng bộ nhớ.
- LGBM sử dụng hai kỹ thuật mới: Gradient-based One Side Sampling (GOSS) và Exclusive Feature Bundling (EFB), đáp ứng các hạn chế của thuật toán dựa trên biểu đồ được sử dụng chủ yếu trong tất cả các khung GBDT (Cây quyết định tăng cường độ dốc). Hai kỹ thuật GOSS và EFB hình thành các đặc điểm của Thuật toán LightGBM. Chúng kết hợp với nhau để làm cho mô hình hoạt động hiệu quả và cung cấp cho nó một lợi thế vượt trội so với các khuôn khổ GBDT khác.
- LightGBM tách cây theo cấp độ cây trái ngược với các thuật toán thúc đẩy khác phát triển theo cấp độ cây. Nó chọn lá bị rụng tối đa để phát triển. Vì lá được cố định nên thuật toán theo lá có tổn thất thấp hơn so với thuật toán theo cấp. Sự phát triển của cây khôn có lá có thể làm tăng độ phức tạp của mô hình và có thể dẫn đến việc trang bị quá nhiều trong các tập dữ liệu nhỏ. Dưới đây là biểu diễn sơ đồ về sự phát triển của cây có lá:
>![alt](https://media.geeksforgeeks.org/wp-content/uploads/20200520035248/Leaf-Wise-Tree-Growth.png)


In [ ]:
from lightgbm import LGBMRegressor
lgbm_reg = LGBMRegressor()
lgbm_preds, y_test = run_model(model=lgbm_reg, matrix_list=sparse_matrix_list)

In [ ]:
print("rmsle: "+str(rmsle(np.expm1(y_test), np.expm1(lgbm_preds))))

**Đánh giá:**
- Qua thử nghiệm 2 mô hình huấn luyện ở trên ta thấy rằng mô hình Ridge Regression có giá trị rmlse nhỏ hơn LGBM 

=> sử dụng mô hình Ridgr Regression

# 12. Dự đoán
Dự đoán giá trên tập test

In [ ]:
sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)
X_train = hstack(sparse_matrix_list).tocsr()
X_train

In [ ]:
y_train = np.log1p(train['price'])
y_train

In [ ]:
Ridge_model =  Ridge()
Ridge_model.fit(X_train, y_train)

In [ ]:
sparse_matrix_list = (test_name, test_des, test_brand_name, test_condition,
                      test_shipping, test_cat_1, test_cat_2, test_cat_3)
X_test = hstack(sparse_matrix_list).tocsr()

In [ ]:
preds = Ridge_model.predict(X_test)
preds

In [ ]:
preds = np.expm1(preds)
preds

# Submission

In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission

In [ ]:
submission.loc[:, 'price'] = preds
submission

In [ ]:
submission.to_csv('submission.csv', index=False)